# Analyze Texts

This notebook combines code by Vi Mai and Moacir P. de Sá Pereira with default TDMStudio code provded by ProQuest/Clarivate.
Mai wrote the Huggingface interface and Sá Pereira put everything together and wrote the general workflow.

This notebook assumes the existence of two different datasets: 

1. A set of corpora available as various directories like `./data/{corpus_name}`, each of which contains $n$ xml files of the name `{goid}.xml`, where `goid` is a global id used by ProQuest for their articles

2. A set of csvs in `./dataframe_files`, each of the name `{corpus}_nnn.csv`. The csvs are each up to 10,000 articles long and were prepared in the `prepare-texts` notebook. The csvs have the following columns:

    - `goid`: Int. As above
    - `title`: Str. The headline of the article
    - `date`: Str. The publication date, in `YYYY-MM-DD` format
    - `publisher`: Str. The article's publisher
    - `pub_title`: Str. The title of the publication
    - `author`: Str. The display name of the author, when available
    - `tokens`: Int. A naive word count, derived from splitting the full text on whitespace.
    
The goal of this notebook is to analyze the sentiment of each article's headline and full text, using different models downloaded from Huggingface. The scores are then added, article-by-article, to the dataframes which are saved as new csvs named `./analyzed_files/{corpus}_nnn.csv`.

At the end, a single parquet file is produced for each corpus (`{corpus}.parquet`) to facilitate moving to the financial modeling section.
    

## Imports

In [13]:
%conda update -n base -c conda-forge conda

Channels:
 - conda-forge
 - nvidia
 - pytorch
Platform: linux-64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [14]:
%conda install pandas=2.2.3

Channels:
 - conda-forge
 - nvidia
 - pytorch
Platform: linux-64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [15]:
%conda install pytorch=2.5

Channels:
 - conda-forge
 - nvidia
 - pytorch
Platform: linux-64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [16]:
%conda install transformers

Channels:
 - conda-forge
 - nvidia
 - pytorch
Platform: linux-64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [17]:
%conda install lxml

Channels:
 - conda-forge
 - nvidia
 - pytorch
Platform: linux-64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [18]:
import os
import random
import pandas as pd
import numpy as np
from lxml import etree
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import torch
import time
import datetime
from transformers import AutoTokenizer, AutoModelForSequenceClassification

## Constants

In [19]:
corpora = [
    "dollar-tree",
    "lululemon",
    "ulta",
    "walgreens",
    "walmart"
]

root_path = "/home/ec2-user/SageMaker"
dataframe_path = f"{root_path}/dataframe_files"
analyzed_csv_path = f"{root_path}/analyzed_files"
analyzed_parquet_path = f"{root_path}/analyzed_parquet_files"

dataframe_files = os.listdir(dataframe_path)

## Subcorpus Construction

In [20]:
def concat_csvs(corpus, token_cutoff_percentage=0.8, omit_weekends=True):
    df = pd.DataFrame()
    for file in dataframe_files:
        if corpus in file:
            df_chunk = pd.read_csv(f"{dataframe_path}/{file}", index_col=0)
            df = pd.concat([df, df_chunk], ignore_index=True)
            df.reset_index(drop=True, inplace=True)
    df["corpus"] = corpus # this lets us know what corpus we are using in later code.
    df["input_tokens"] = 0 # this number will be replaces if an article is analyzed
    
    # Calculate the token breakpoint to remove outliers in terms of token counts.
    # Some articles are unrealistically long (several thousands of tokens) and are
    # likely aggregations of all kinds of information.    
    cutoff = df["tokens"].quantile(token_cutoff_percentage)
    df = df[df["tokens"] < cutoff]
    
    if omit_weekends:
        # Remove all articles that were published on weekends
        df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d")
        df = df[df["date"].dt.dayofweek < 5]
    
    # Build up aggregate columns
    date_group = df.groupby("date")
    df["daily_article_count"] = date_group.transform("size")
    df["daily_token_sum"] = date_group["tokens"].transform("sum")
    
    return df

## Text Analysis Functions

In [21]:
# Function to read a single text from a single xml file.
def get_text(corpus, goid):
    text = ""
    try: 
        tree = etree.parse(f"{root_path}/data/{corpus}/{goid}.xml")
        root = tree.getroot()
        if root.find('.//FullText') is not None:
            text = root.find('.//FullText').text
        elif root.find('.//HiddenText') is not None:
            text = root.find('.//HiddenText').text
        elif root.find('.//Text') is not None:
            text = root.find('.//Text').text
        
        text = BeautifulSoup(text).get_text().replace('\n', ' ').replace('\\', '').strip()

    except Exception as e:
        print(f"Error while parsing file {file}: {e}")
        
    return text

In [26]:
class SentimentAnalysisEngine:
    def __init__(self, huggingface_model = "distilroberta-finetuned-financial-news-sentiment-analysis"):
        self.model_path = f"models/{huggingface_model}"
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path, local_files_only = True)
        self.model = AutoModelForSequenceClassification.from_pretrained(self.model_path, local_files_only = True)
        self.max_length = 512 # Huggingface maximum
        
    def analyze(self, text):
        # Adapted from https://github.com/huggingface/transformers/issues/9321
        averages = []
        errors = []
        chunk_sizes = []
        
        inputs = self.tokenizer(text, return_tensors="pt")
        input_ids = inputs["input_ids"][0]
        length = len(input_ids)
        
        # Chunk input tensor into max_length-sized pieces.
        chunks = [
            input_ids[i:i + self.max_length]
            for i in range(0, length, self.max_length)
        ]
        
        for chunk in chunks:
            chunk_inputs={k: v[:, :len(chunk)].reshape(1, -1) for k, v in inputs.items()}
            chunk_inputs["input_ids"] = chunk.reshape(1, -1)
        
            preds = self.model(**chunk_inputs)
            weights = torch.softmax(preds.logits, dim=1) # could move to cuda in theory
            # print(torch.allclose(torch.sum(weights, dim=-1), torch.tensor(1.0)))  # Returns True if the sum is 1
            values = torch.linspace(-1, 1, steps=3)
            average = torch.dot(weights[0], values) # Don't need to divide. Weights sum to 1 in softmax
            averages.append(average.item())
            
            deviations = (values - average) ** 2
            variance = torch.dot(weights[0], deviations)
            error = torch.sqrt(variance)
            errors.append(error.item())
            
            chunk_sizes.append(len(chunk))
        
        chunk_sizes = torch.tensor(chunk_sizes, dtype=torch.float32)
        chunk_weighted_averages = torch.tensor(
            averages, dtype=torch.float32) * chunk_sizes
        errors = torch.tensor(errors, dtype=torch.float32)
        inverse_squared_errors = 1 / errors**2
        chunk_weighted_errors = inverse_squared_errors * chunk_sizes
        weighted_average = torch.sum(
            chunk_weighted_averages * inverse_squared_errors
            ) / torch.sum(chunk_weighted_errors)
        weighted_error = torch.sqrt(1.0 / torch.sum(1.0 / chunk_weighted_errors))
            
        # Return the final weighted average
        return weighted_average.item(), weighted_error.item(), length # total number of tokens

#         while length > 0:
#             if length > self.max_length:
#                 next_inputs={k: (i[0][self.max_length:]).reshape(1,len(i[0][self.max_length:])) for k, i in inputs.items()}
#                 inputs={k: (i[0][:self.max_length]).reshape(1,len(i[0][:self.max_length])) for k, i in inputs.items()}
#             else:
#                 next_inputs=False
#             preds = self.model(**inputs)
#             weights = torch.softmax(preds[0], dim=1)
#             np_weights = weights.detach().numpy()[0]
# #            list_weights = list(weights.detach().numpy()[0])
# #             weights = proOrCon[0].detach().numpy()[0]
# #             weights[2], weights[1] = weights[1], weights[2]
# #             weights = softmax(weights)
#             average = np.average(np.linspace(1, -1, 3), weights = np_weights)
#             averages.append(average)
#             errors.append(
#                 np.sqrt(np.average(np.array(np.linspace(1, -1, 3)-average)**2, weights = np_weights))
#             )
#             if next_inputs:
#                 inputs=next_inputs
#             else:
#                 break
#             length=len(inputs['input_ids'][0])
#         average = np.average(averages, weights = 1./np.array(errors)**2)
#         error = np.sqrt(1./np.sum(1./np.array(errors)**2))
# #         print(error)
# #         print(average) #, error)
# #         # Prediction
# #         preds = self.model(tok_inp['input_ids'])
# #         return self.classes[torch.argmax(preds.logits)]
#         return float(average)
                
#             # Compute the weighted average using PyTorch
#             values = torch.linspace(1, -1, steps=3, device=weights.device)
#             average = torch.dot(weights[0], values)
#             averages.append(average.item())
        
#             # Compute the standard deviation (error) using PyTorch
#             deviations = (values - average) ** 2
#             variance = torch.dot(weights[0], deviations)
#             error = torch.sqrt(variance)
#             errors.append(error.item())  
#             if next_inputs:
#                 inputs = next_inputs
#             else:
#                 break
        
#             length = len(inputs['input_ids'][0])
#         # Compute final weighted average and error using PyTorch
#         weights = torch.tensor([1.0 / (e ** 2) for e in errors], device=weights.device)
#         final_average = torch.sum(torch.tensor(averages, device=weights.device) * weights) / torch.sum(weights)
#         # Figure out what to do with error later.
#         final_error = torch.sqrt(1.0 / torch.sum(weights))
    
#         return float(final_average)

In [27]:
sentiment_analyzer = SentimentAnalysisEngine()

In [28]:
sentiment_analyzer.analyze("This stock is going to crash very soon")
# -0.9898354917397236

(-0.9898356199264526, 30.163711547851562, 10)

In [29]:
sentiment_analyzer.analyze("This stock is going to soar very soon")
# 0.9984612356163491

(0.9984613060951233, 69.96183013916016, 10)

In [30]:
sentiment_analyzer.analyze("This stock is going to blah")
# -3.935034447049273e-05

(-3.9350346924038604e-05, 237.02548217773438, 8)

## Iterate

In [ ]:
sentiment_analyzer = SentimentAnalysisEngine()

def analyze_row(row):
    text = get_text(row.corpus, row.goid) # Why we added the `corpus` column.
    title_sentiment = float(sentiment_analyzer.analyze(row.title))
    text_sentiment = float(sentiment_analyzer.analyze(text))
    return title_sentiment, text_sentiment

In [ ]:
sample_count = 500

start_time = time.time()

now = datetime.datetime.now()
print(now)

tqdm.pandas(desc="Progress")

for corpus in corpora:
    df = concat_csvs(corpus)
    sample_df = df.sample(n=sample_count, random_state=42)
    sample_df[['title_sentiment', 'text_sentiment']] = sample_df.progress_apply(lambda row: analyze_row(row), axis=1, result_type="expand")
    file_path = f"{analyzed_csv_path}/{corpus}.csv"
    sample_df.to_csv(file_path)
    print(f"Wrote {file_path}")
    
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Iterate


for corpus in corpora:
    for file in output_files:
        if corpus in file:
            df = pd.read_csv(f"{csv_path}/{file}")
            # Add new columns
            
            #token_breakpoint[corpus]
    
            file_path = f"{analyzed_csv_path}/{file}"
            df.to_csv(file_path)
            print(f"Wrote {file_path}")

In [ ]:
# parquet



for corpus in corpora:
    file_path = f"{analyzed_parquet_path}/{corpus}.parquet"
    df = pd.DataFrame()
    for file in output_files:
        if corpus in file:
            df_chunk = pd.read_csv(f"{analyzed_csv_path}/{file}")
            df = pd.concat([df, df_chunk], ignore_index=True)
    df.to_parquet(file_path)
    print(f"Wrote {file_path}")


In [ ]:
n = np.linspace(1, -1, 3)

In [ ]:
np.average(n)

In [ ]:
np.__version__

In [ ]:
torch.__version__

In [31]:
!aws s3 cp ./analyze-texts.ipnyb s3://pq-tdm-studio-results/tdm-ale-data/1876/results/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



The user-provided path ./analyze-texts.ipnyb does not exist.
